In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pathlib import Path

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

sys.path.append("/home/acho/Sync/Python/sigtools")
from sigtools.representations import *
# from sigtools.sounds import *
# from sigtools.processing import *
# from sigtools.spatialization import *


PROJ_DIR = Path.home()/"Sync"/"KiddLab"/"MSM"
STIM_DIR = PROJ_DIR/"assets"/"stimuli"
# STIM_DATABASE = pd.read_csv(STIM_DIR/"stimulus_database.csv")

INITIALIZATION
Setting paths...
Importing sigtools...
Setting up sounddevice...
Importing level adjustments...
Setting eligible BUG words...
Set values for tone pattern synthesis...
Initialization complete!


In [31]:
CFs, p, snd = make_TP_sequence(3, 5, 2)
print(CFs)
print(p)
(normalize_rms([sum(center_sounds(snd))])[0] - 20).play()

[ 336  820 1282]
[['CONSTANT' 'FALLING' 'ALTERNATING' 'STEP-DOWN' 'CONSTANT']
 ['STEP-DOWN' 'ALTERNATING' 'STEP-DOWN' 'CONSTANT' 'STEP-UP']
 ['FALLING' 'STEP-UP' 'RISING' 'RISING' 'STEP-DOWN']]
